Customized Transfer Learning (TL) Method for handwritten digit classification using VGG16 

Part (a): Generate a TL Model with VGG16
Freezing Fully-Connected (FC) and Output Layers of the VGG16 model.
Set a new FC layer and an output layer for MNIST digit classification.
Initialize the model with pre-trained weights on ImageNet.

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import mnist
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Load the MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Normalize the dataset and resize to 32x32
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0
x_train_resized = tf.image.resize(x_train[..., np.newaxis], (32, 32))
x_test_resized = tf.image.resize(x_test[..., np.newaxis], (32, 32))

In [ ]:
# Convert single channel grayscale to 3-channel for VGG16 compatibility
x_train_resized = np.repeat(x_train_resized, 3, axis=-1)
x_test_resized = np.repeat(x_test_resized, 3, axis=-1)

In [ ]:
# Load the VGG16 model without the top layers, freezing its convolutional layers
vgg16_base = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))
for layer in vgg16_base.layers:
    layer.trainable = False

In [ ]:
# Add new fully-connected layers
x = Flatten()(vgg16_base.output)
x = Dense(512, activation='relu')(x)  # Fully-connected layer with 512 units
output = Dense(10, activation='softmax')(x)  # Output layer for 10 classes

In [ ]:
# Create the new model
custom_vgg16_model = Model(inputs=vgg16_base.input, outputs=output)
custom_vgg16_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [ ]:
# Display the customized VGG16 model
custom_vgg16_model.summary()

In [ ]:
# Data Augmentation
datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2
)


In [ ]:
datagen.fit(x_train_resized)

In [ ]:
# Train on original MNIST dataset
history_original = custom_vgg16_model.fit(x_train_resized, y_train, validation_split=0.1, epochs=10, batch_size=32)


In [ ]:
# Train on augmented MNIST dataset
history_augmented = custom_vgg16_model.fit(datagen.flow(x_train_resized, y_train, batch_size=32),
                                           validation_data=(x_test_resized, y_test), epochs=10)



# Evaluate on test data
test_loss_orig, test_acc_orig = custom_vgg16_model.evaluate(x_test_resized, y_test)
print(f"Test accuracy on original data: {test_acc_orig:.4f}")

# Evaluate on augmented test data
test_loss_aug, test_acc_aug = custom_vgg16_model.evaluate(x_test_resized, y_test)
print(f"Test accuracy on augmented data: {test_acc_aug:.4f}")

In [ ]:
# Plot accuracy comparison
plt.plot(history_original.history['accuracy'], label='Original Training Accuracy')
plt.plot(history_original.history['val_accuracy'], label='Original Validation Accuracy')
plt.plot(history_augmented.history['accuracy'], label='Augmented Training Accuracy')
plt.plot(history_augmented.history['val_accuracy'], label='Augmented Validation Accuracy')
plt.title('Training and Validation Accuracy of VGG16 Models')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()